In [4]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
import jieba
import jieba.posseg as pseg
import jieba.analyse
from optparse import OptionParser
import pandas as pd
import csv
from pyecharts.charts import WordCloud
!pip install --upgrade pip
import seaborn as sns
import wordcloud
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.font_manager import _rebuild

Requirement already up-to-date: pip in ./venv/lib/python3.7/site-packages (20.1.1)


In [5]:
jieba.enable_paddle()

Paddle enabled successfully......


In [6]:
import collections

#set stopwords
def stopwordslist(filepath):   
    stopwords = [line.strip() for line in open(filepath, 'r').readlines()]  
    return stopwords #return a list

stopwords = stopwordslist('/Users/zhaoxiaozhao/Downloads/stopwords.txt')
stopwords = set(stopwords)

# print(stopwords)
def remove_stopwords(words): #remove stopwords
    processed_word_list = []
    for word in words:
        word = word.lower() # in case they are not all lower cased
        if word not in stopwords:
            processed_word_list.append(word)
    return processed_word_list

def word_list(filename):
    #Input your own file location(text location)
    with open('/Users/zhaoxiaozhao/Downloads/'+ filename, 'r') as file:
        data = file.read().replace('\n', '')
        test_ls = jieba.lcut(data,cut_all=False)
        test_ls = remove_stopwords(test_ls)
    return test_ls


def cnt_words_except_sin(filename):
    #Input your own file location(where to store)
    with open('/Users/zhaoxiaozhao/Downloads/'+ filename, 'r') as file:
        data = file.read().replace('\n', '')
        #使用 jieba.lcut
        test_ls_txt = jieba.lcut(data,cut_all=False)
        test_ls_txt = remove_stopwords(test_ls_txt)   
        dict_words_frequency={}
        
    for n in test_ls_txt:
        if len(n) != 1:
            dict_words_frequency[n]=test_ls_txt.count(n)
        
    cnt = len(dict_words_frequency) #count all words
    print(cnt)
    #sort the dictionary
    sorted_x = sorted(dict_words_frequency.items(), key=lambda kv: kv[1])
    dict_words_frequency = collections.OrderedDict(sorted_x)
    
    df = pd.DataFrame.from_dict(dict_words_frequency, orient='index', columns=['count'])
    df = df.sort_values(by = ['count'], ascending=True)
    # new column for 词频（百分比）
    df['q'] = df['count'] / cnt

    #return DataFrame(type)
    return df

# Return 分词 + 词性标注
def cnt_words_flag(filename):
    #Input your own file location(where to store)
    with open('/Users/zhaoxiaozhao/Downloads/'+ filename, 'r') as file:
        data = file.read().replace('\n', '')
        #使用 pseg.lcut
        test_ls = pseg.lcut(data,use_paddle=True)  
        dict_words_flags = {}

    for i in test_ls:
        dict_words_flags[i.word] = i.flag

    #return DataFrame(type)
    df = pd.DataFrame.from_dict(dict_words_flags, orient='index', columns=['flag'])
    
    #Input your file name
    to_csv(df,'hhh.csv')
    return df

# sort the df considering 词频（百分比）
def sort_q(df):
    df = df.sort_values(df.columns[1], ascending=False)
    return df

# return 2-pair array list for 词云
def ls_cnt_words(filename):
    with open('/Users/zhaoxiaozhao/Downloads/'+ filename, 'r') as file:
        data = file.read().replace('\n', '')
        test_ls = jieba.lcut(data,cut_all=False)
        test_ls = remove_stopwords(test_ls)
        dict_words_frequency={}
    for n in test_ls:
        dict_words_frequency[n]=test_ls.count(n)
    sorted_x = sorted(dict_words_frequency.items(), key=lambda kv: kv[1])
    dict_words_frequency = collections.OrderedDict(sorted_x)
    words = list(dict_words_frequency.keys())
    value = list(dict_words_frequency.values())   
    
    new_ls = []
    for i in range(len(words)):
        new_ls.append((words[i],value[i])) 
    x = pd.Series(dict_words_frequency)
    x = x.sort_values(ascending=False)
    print(x.describe())
    return new_ls

def ls_q_words(df):
#     for col in df.columns:
    df = df[df.columns]
    words = df.index.tolist()
    value = df[df.columns[1]].tolist()
    
    new_ls = []
    for i in range(len(words)):
        new_ls.append((words[i],value[i]))
    
    return new_ls


def word_count(processed_word_list):
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.rcParams['axes.unicode_minus'] = False
    word_count = pd.Series(processed_word_list).value_counts().sort_values(ascending=False)[0:15]  
    fig = plt.figure(figsize=(16,8))  
    x = word_count.index.tolist()  
    y = word_count.values.tolist()  
    sns.barplot(x, y, palette="BuPu_r")  
    plt.title('word frequency top 15')  
    plt.ylabel('count')  
    sns.despine(bottom=True)  
    #plt.savefig('./word_count_bar.png',dpi=400)  
    plt.show()


def to_csv(x,filename):
    
    Frame=pd.DataFrame(x,columns = ['cnt'])
    x.to_csv('/Users/zhaoxiaozhao/Downloads/'+filename, encoding='utf-8')

    
_rebuild()


to_csv(cnt_words_except_sin('detail_zh.txt'),'detail_zh.csv')

test_ls = ls_q_words(cnt_words_except_sin('detail_zh.txt'))
cnt_words_flag('detail_zh.txt')

from PIL import Image
import wordcloud
import numpy as np


wordcloud = WordCloud()
wordcloud.add("", test_ls, word_size_range=[20, 100])
wordcloud.render('detail_zh-wordcloud.html')

from IPython.display import IFrame
IFrame('detail_zh-wordcloud.html', width=700, height=520)



Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/1j/d3xd3shn42x71_yhgx8757r80000gn/T/jieba.cache
Loading model cost 0.757 seconds.
Prefix dict has been built successfully.


4146
4146
